In [66]:
# Importa a função para carregar pastas de trabalho
from openpyxl import load_workbook 

# 1. Abrir a planilha
# Substitua 'seu_arquivo.xlsx' pelo nome do seu arquivo.
try:
    workbook = load_workbook(r'C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc\eda.xlsx')
except FileNotFoundError:
    print("Erro: Arquivo 'seu_arquivo.xlsx' não encontrado.")
    # Você pode adicionar aqui a lógica para criar o arquivo se ele não existir,
    # usando from openpyxl import Workbook e wb = Workbook().
    # Para este exemplo, apenas saímos.
    exit()

# 2. Selecionar a Planilha (Sheet)
# Você pode usar o nome da planilha ou a planilha ativa
# Se o nome for 'Planilha1', use:
sheet = workbook['Planilha1'] 

# Se quiser usar a planilha ativa (geralmente a primeira):
# sheet = workbook.active 

# 3. Alterar o Conteúdo de uma Célula

# Há duas formas principais de fazer referência à célula:
# A) Usando notação de coluna/linha do Excel (ex: 'A1', 'B5'):
sheet['W2'] = '77.12'

# B) Usando indexação por linha e coluna (índice começa em 1):
# A célula na linha 2, coluna 3 (que é a célula C2) receberá 'Alterado!'
sheet.cell(row=2, column=24, value='23,88')

# 4. Salvar a planilha
# Você pode salvar no mesmo arquivo ou em um novo.
# Se salvar no mesmo arquivo, ele será sobrescrito.
workbook.save(r'C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc\eda.xlsx')

print("FIM !")

FIM !


In [ ]:
from math import nan
import pandas as pd
from charset_normalizer import from_path 
import re 
from pandas import DataFrame, Series
from typing import Dict, Any, Tuple, Optional, Union


In [ ]:

config_path = r"C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc"
config_file = r"\eda.xlsx"
result_path = r"C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc"
result_file = r"\eda_result.xlsx"
path_dados  = r"C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\data"

sheet_file = config_path+config_file
sheet_fields = ('tab_nome','tab_descricao','tab_arquivo','tab_tamanho','tab_linhas','tab_colunas','tab_tipo','cpo_nome','cpo_desc','cpo_tipo','cpo_tamanho','cpo_formato','cpo_pk','cpo_fk','cpo_tab_fk','cpo_vazio','cpo_valores','cpo_limite','cpo_regex','rsp_vazio','rsp_unico','rsp_repetido','rsp_formato','rsp_invalido','rsp_limite','rsp_min','rsp_max','rsp_referencial')
df_eda_sheet = pd.read_excel(sheet_file,header=None, skiprows=1, names=sheet_fields)
df_eda_sheet.columns = df_eda_sheet.columns.str.lower() 
df_eda_sheet["cpo_limite"] = df_eda_sheet["cpo_limite"].fillna('')
df_eda_sheet["cpo_limite"] = df_eda_sheet["cpo_limite"].astype(str)

nome_tabela = ""
nome_tabela_atual = ""
for index, sheet_row in df_eda_sheet.iterrows():

    if (not pd.isna(sheet_row['tab_nome']) and not sheet_row['tab_nome'] == "Tabela"): 
        nome_tabela = sheet_row['tab_nome']

        # Le os dados 
        nome_arquivo = sheet_row['tab_arquivo']
        file_path = path_dados + '/' + nome_arquivo
        encode = ""
        separador = ";"
        try:
            if len(encode) == 0: 
               detectado = from_path(str(file_path)).best() 
               encoding_detectado = detectado.encoding 
            else: 
               encoding_detectado = encode
        except Exception:
            encoding_detectado = 'utf-8' 
        # data Load    
        df_dados = pd.read_csv(file_path,encoding=encoding_detectado,sep=separador,engine='python')
        df_dados.columns = df_dados.columns.str.lower() 

    if (pd.isna(sheet_row['cpo_nome']) or sheet_row['cpo_nome'] == "Campo"): 
        continue 
    
    total_registros = len(df_dados)
    nome_campo = sheet_row['cpo_nome']
    # -----------------------------------------------------
    # Vazios: Brancos, nulos e zerados
    # -----------------------------------------------------

    percentual_problema = 0 
    cond_nulos = pd.isna(df_dados[nome_campo])

    cond_vazios = (df_dados[nome_campo].astype(str).str.strip() == '')
    cond_zeros = (df_dados[nome_campo] == 0)
    cond_zeros_str = (df_dados[nome_campo] == 0) | (df_dados[nome_campo] == '0')
    mascara_total = cond_nulos | cond_vazios | cond_zeros
    percentual_problema = mascara_total.mean() 
    df_eda_sheet.loc[index, 'rsp_vazio'] = percentual_problema

    # -----------------------------------------------------
    # Valores Únicos
    # -----------------------------------------------------
    contagem_unicos = df_dados[nome_campo].nunique()
    prcentual_unicos = (contagem_unicos / total_registros) 
    df_eda_sheet.loc[index, 'rsp_unico'] = prcentual_unicos

    # -----------------------------------------------------
    # Valores Repetidos
    # -----------------------------------------------------
    mascara_repetidos = df_dados[nome_campo].duplicated(keep=False)
    contagem_repetidos = mascara_repetidos.sum()
    percentual_repetidos = (contagem_repetidos / total_registros)
    df_eda_sheet.loc[index, 'rsp_repetido'] = percentual_repetidos

    # -----------------------------------------------------
    # Formato (Regex)
    # -----------------------------------------------------
    regex = sheet_row['cpo_regex']
    coluna_limpa = df_dados[nome_campo].astype(str).str.replace(r'\.0', '', regex=True).str.strip()
    mascara_sucesso = coluna_limpa.str.match(str(regex), na=False)
    mascara_falha = ~mascara_sucesso
    proporcao_sucesso = mascara_falha.mean()
    df_eda_sheet.loc[index, 'rsp_formato'] = proporcao_sucesso

    # -----------------------------------------------------
    # Valores inválidos em uma lista 
    # -----------------------------------------------------
    lista_valores = sheet_row['cpo_valores']
    if not pd.isna(lista_valores): 
        lista_validos = [item.strip().upper() for item in lista_valores.split(';')]
        coluna_para_validar = df_dados[nome_campo].astype(str).str.upper()
        mascara_sucesso = coluna_para_validar.isin(lista_validos)        
        mascara_falha = ~mascara_sucesso
        percentual_falha = mascara_falha.mean()
        df_eda_sheet.loc[index, 'rsp_invalido'] = percentual_falha
    else: 
        df_eda_sheet.loc[index, 'rsp_invalido'] = ""

    # -----------------------------------------------------
    # Limites de valores 
    # -----------------------------------------------------
    limite_string = sheet_row['cpo_limite']
    if pd.isna(limite_string): 

        limites = re.findall(r'\d+', limite_string)
        if len(limites) != 2:
            print(f"ERRO: Formato de limite inválido ({limite_string}). Esperado 'MÍNIMO MÁXIMO'.")
            percentual_erro = "" 
        else:
            # Converte os limites para FLOAT (ponto flutuante)
            try:
                limite_min = float(limites[0])
                limite_max = float(limites[1])
            except ValueError:
                print(f"ERRO: Limites encontrados ({limites}) não são números válidos.")
                percentual_erro = ""
        coluna_numerica = pd.to_numeric(df_dados[nome_campo], errors='coerce')
        mascara_validacao = pd.notna(coluna_numerica)
        total_para_validar = mascara_validacao.sum()
        if total_para_validar == 0:
            percentual_erro = 0.0
        else:
            cond_min = (coluna_numerica >= limite_min) 
            cond_max = (coluna_numerica <= limite_max)
            mascara_falha_limite = (~(cond_min & cond_max)) & mascara_validacao
            percentual_erro = (mascara_falha_limite.sum() / total_para_validar) 
        df_eda_sheet.loc[index, 'rsp_limite'] = percentual_erro

    # -----------------------------------------------------
    # Valores Minimo e Maximo da coluna
    # -----------------------------------------------------
    coluna_numerica = pd.to_numeric(df_dados[nome_campo], errors='coerce')
    valor_minimo = coluna_numerica.min()
    valor_maximo = coluna_numerica.max()
    if not pd.isna(valor_minimo):
        df_eda_sheet.loc[index, 'rsp_min'] = valor_minimo
    if not pd.isna(valor_maximo): 
        df_eda_sheet.loc[index, 'rsp_max'] = valor_maximo

    # -----------------------------------------------------
    # Integridade referencial
    # -----------------------------------------------------
    # Abre a tabela relacionada e procura pela FK  
    #rsp_relacional 'cpo_fk','cpo_tab_fk'
    if sheet_row['cpo_fk'] == "Sim": 
        nome_tabela_relacionada = sheet_row['cpo_tab_fk']
        file_path_fk = path_dados + '/' + nome_tabela_relacionada + ".csv"
        encode = "windows-1252"
        separador = ";"
        try:
            if len(encode) == 0: 
                detectado = from_path(str(file_path)).best() 
                encoding_detectado = detectado.encoding 
            else: 
                encoding_detectado = encode
        except Exception:
            encoding_detectado = 'utf-8'  
        df_dados_fk = pd.read_csv(file_path_fk,encoding=encoding_detectado,sep=separador,engine='python')
        df_dados_fk.columns = df_dados_fk.columns.str.lower() 
        chaves_primarias_validas = set(df_dados_fk[nome_campo].dropna().unique())
        registros_a_checar = df_dados[nome_campo].dropna()
        total_registros_checado = len(registros_a_checar)
        if total_registros_checado == 0:
            percentual_integridade = 0.0
        else:
            mascara_sucesso_relacional = registros_a_checar.isin(chaves_primarias_validas)
            proporcao_sucesso = mascara_sucesso_relacional.mean()
            percentual_integridade = proporcao_sucesso 
            df_eda_sheet.loc[index, 'rsp_referencial'] = percentual_integridade
        
    # Salva a planilha de saida
    df_eda_sheet.to_excel(result_path+result_file, sheet_name='resultados_eda', index=False,)

